In [ ]:
# modified from https://github.com/keon/deep-q-learning/blob/master/cartpole.py

import copy
from collections import deque

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
import numpy as np

from human_play_pong import MyPongEnv

episodes = 1000

class DQNAgent:
  def __init__(self, env):
    self.env = env
    self.memory = deque(maxlen=10000)
    self.gamma = 0.9  # decay rate
    self.epsilon = 0.7  # exploration
    self.epsilon_decay = .99
    self.epsilon_min = 0.05
    self.learning_rate = 0.0001
    self._build_model()

  def _build_model(self):
    # Deep-Q learning Model
    model = Sequential()
    model.add(Dense(64, input_dim=self.env.STATE_SIZE, activation='tanh', init='he_uniform'))
    model.add(Dense(128, activation='tanh', init='he_uniform'))
    model.add(Dense(128, activation='tanh', init='he_uniform'))
    model.add(Dense(2, activation='linear', init='he_uniform'))
    model.compile(loss='mse', optimizer=RMSprop(lr=self.learning_rate))
    self.model = model

  def remember(self, state, action, reward, next_state):
    self.memory.append((state, action, reward, next_state))

  def act(self, state):
    if np.random.rand() <= self.epsilon:
      return self.env.sample_action_space()
    act_values = self.model.predict(state)
    max_action = np.argmax(act_values[0])
    ret_array = np.zeros((3,), dtype=int)
    ret_array[max_action] = 1
    return ret_array  # returns action

  def replay(self, batch_size):
    batchs = min(batch_size, len(self.memory))
    batchs = np.random.choice(len(self.memory), batchs)
    for i in batchs:
      state, action, reward, next_state = self.memory[i]
      target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
      target_f = self.model.predict(state)
      target_f[0][action] = target
      self.model.fit(state, target_f, nb_epoch=1, verbose=0)
    if self.epsilon > self.epsilon_min:
      self.epsilon *= self.epsilon_decay


env = MyPongEnv()
agent = DQNAgent(env)

# run an episode
for e in range(episodes):
  state = env.reset()
  state = np.reshape(state, [1, 3])
  
  # run for 5000 time steps per episode
  for time_t in range(5000):
    env.render()
    action = agent.act(state)
    next_state, reward, done = env.step(action)
    next_state = np.reshape(next_state, [1, 3])
    reward = -100 if done else reward
    agent.remember(state, action, reward, next_state)
    state = copy.deepcopy(next_state)
    if done:
      print("episode: {}/{}, score: {}, memory size: {}, e: {}"
            .format(e, episodes, time_t, len(agent.memory), agent.epsilon))
      break # if done go to next episode
    agent.replay(32)

Using TensorFlow backend.
